In [1]:
from langchain_community.document_loaders import (
    PythonLoader,
    DirectoryLoader,
    TextLoader,
)
import os, json
from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings
from langchain_cohere.embeddings import CohereEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.agent_toolkits import FileManagementToolkit

groq_api_key = "your Groq api key"

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    temperature=0.0,
)


embeddings_obj = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key="cdu2whMVXxqmnCTG8wAIqd35ghLPgCnCKG7b8liN",
)


# loader=TextLoader('summary_all.txt')


# text_data=loader.load()


# text_data

In [2]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
# text_splitted = text_splitter.split_documents(text_data)

In [3]:
# from langchain_community.vectorstores import FAISS

# db=FAISS.from_documents(text_splitted,embeddings_obj)

In [4]:
from langchain_community.vectorstores import FAISS

db = FAISS.load_local(
    "Deep_learning_Java_Codes", embeddings_obj, allow_dangerous_deserialization=True
)
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBC0A5FC90>)

In [5]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "Vector_database",
    description=(
        """This tool retrieves documents file from a vector database based on the user's query. 
        Analyze the content of the user's query and match it with the most relevant documents in the database. 
        If the content and context of the query closely align with the documents' subject matter, 
        provide the corresponding file path where the matching document is stored by using the context passed in retriever object"""
    ),
)

In [6]:
from langchain import hub


prompt_react = hub.pull("hwchase17/react-chat")
prompt_react

c:\Users\Rudra\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat', 'lc_hub_commit_hash': '3ecd5f710db438a9cf3773c57d6ac8951eefd2cd9a9b2a0026a65a0893b86a6e'}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. A

In [7]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool(
    name="Intermediate Answer",
    description="This tool is used for the commanding into windows os terminal and getting back the results of all types (eg.'grep','cls')",
    handle_tool_error=True,
    handle_validation_error=True,
    response_format="content",
)

In [8]:
import os
from langchain.tools import Tool

def find_files_by_name(filename: str):
    root_dir = "C:\\Users\\Rudra\\Desktop"
    matches = []

    def search_directory(directory):
        with os.scandir(directory) as entries:
            for entry in entries:
                if entry.is_file() and entry.name == filename:
                    matches.append(entry.path)
                elif entry.is_dir():
                    search_directory(entry.path)

    search_directory(root_dir)
    if matches!=[]:
        return matches
    else:
        return "Sorry file couldn't found, please try again with some other clues...."

file_finder = Tool.from_function(
    func=find_files_by_name,
    name="file_finder",
    description="A tool for finding the file by their filename in 'C:\\Users\\Rudra\\Desktop' directory takes 2 input directory path and filename as input in it",
)

In [9]:
tools = [retriever_tool, file_finder]
tools

[Tool(name='Vector_database', description="This tool retrieves documents file from a vector database based on the user's query. \n        Analyze the content of the user's query and match it with the most relevant documents in the database. \n        If the content and context of the query closely align with the documents' subject matter, \n        provide the corresponding file path where the matching document is stored by using the context passed in retriever object", args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DBC1A13B00>, retriever=VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBC0A5FC90>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001DBC1A13D80>, retriever=VectorStor

In [10]:
template = """Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}"""
prompt_react = PromptTemplate.from_template(template)

In [11]:
from langchain.agents import create_react_agent
from langchain.chains.conversation.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="history", return_messages=True)
react_agent = create_react_agent(llm, tools, prompt_react)

In [12]:
from langchain.agents import AgentExecutor, AgentType

agent_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    memory=memory,
    max_execution_time=30
)
agent_executor

AgentExecutor(memory=ConversationBufferMemory(return_messages=True), verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "Vector_database(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBC0A5FC90>), document_prompt: 'BasePromptTemplate' = PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator: 'str' = '\\n\\n', callbacks: 'Callbacks' = None) -> 'str' - This tool retrieves documents file from a vector database based on the user's query. \n        Analyze the content of the user's query and match it with the most relevant documents in the database. \n        If the content and context of the query closely align with the do

In [13]:
import warnings

warnings.filterwarnings("ignore")
agent_executor.invoke({"input": "give the list of filepaths of files which contains the code for tower of hanoy problem in java"})["output"]



> Entering new AgentExecutor chain...
I need to find documents that contain the code for the Tower of Hanoi problem in Java. I will use the Vector_database tool to find the most relevant documents.

Action: Vector_database
Action Input: "Tower of Hanoi problem in Java"{'files': [{'filename': 'Tower_Hanoy.java', 'summary': 'Tower of Hanoy puzzle solver in Java, uses

in Java. It uses backtracking to solve the puzzle. It checks each possible number in each empty

in Java, uses recursion, with functions for moving n disks from one tower to another.', 'filepath':

Sort, Merge Sort, Quick Sort, Linear Search, Binary Search, and Tower of Hanoi. It also containsI found some documents that might contain the code for the Tower of Hanoi problem in Java. I will now use the file_finder tool to locate these files.

Action: file_finder
Action Input: "Tower_Hanoy.java"['C:\\Users\\Rudra\\Desktop\\Java\\Tower_Hanoy.java']Final Answer: The file path for the Tower of Hanoi problem in Java is C:\Users\

'The file path for the Tower of Hanoi problem in Java is C:\\Users\\Rudra\\Desktop\\Java\\Tower_Hanoy.java.'